In [1]:
import os
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_dir='F:\BrainCancer_DataSet\Train'
test_dir='F:\BrainCancer_DataSet\Test'
save_dir='F:/BrainCancer_DataSet'

In [2]:
custom_transform=transforms.Compose([transforms.Resize((512,512),interpolation=3),transforms.ToTensor()])

In [3]:
class CNNModel(nn.Module):
    def __init__(self,in_ch,num_cl,hidden_ch=64):
        super(CNNModel,self).__init__()
        self.Model=nn.Sequential(
        self.block(in_ch,hidden_ch,final=False),
        self.block(hidden_ch,hidden_ch*2,final=False),
        nn.MaxPool2d((2,2)),
        self.block(hidden_ch*2,hidden_ch*4,final=False),
        nn.MaxPool2d((2,2)),
        self.block(hidden_ch*4,hidden_ch*8,final=False),
        nn.MaxPool2d((2,2)),
        self.block(hidden_ch*8,hidden_ch*8,final=False),
        self.block(hidden_ch*8,num_cl,final=True)
        )
        
    def block(self,in_ch,out_ch,final):
        if not final:
            return nn.Sequential(
            nn.Conv2d(in_ch,out_ch,kernel_size=5,stride=1,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            )
        else:
            return nn.Sequential(
            nn.Conv2d(in_ch,out_ch,kernel_size=60,stride=1,padding=0),
            nn.Sigmoid()
            )
    def forward(self,x):
        x=self.Model(x)
        return x.reshape(x.shape[0],-1)

In [4]:
x=torch.randn(10,3,512,512)
model=CNNModel(in_ch=3,num_cl=4)
print(model(x).shape)

torch.Size([10, 4])


In [5]:
train_dataset=datasets.ImageFolder(train_dir,transform=custom_transform)
train_data=DataLoader(train_dataset,batch_size=128,shuffle=True)
test_dataset=datasets.ImageFolder(test_dir,transform=custom_transform)
test_data=DataLoader(test_dataset,batch_size=64,shuffle=True)
print(train_dataset)
print(test_dataset)

Dataset ImageFolder
    Number of datapoints: 4456
    Root location: F:\BrainCancer_DataSet\Train
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=PIL.Image.BICUBIC)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 394
    Root location: F:\BrainCancer_DataSet\Test
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=PIL.Image.BICUBIC)
               ToTensor()
           )


In [6]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=.001)
epoch=5
def accuracy(data,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for images,labels in tqdm(data):
            images=images.to(device)
            labels=labels.to(device)
            preds=model(images)
            values,index=preds.max(1)
            correct+=(index==labels).sum()
            total+=preds.size(0)
            acc=correct/total
        print(acc)

In [ ]:
for e in range(epoch):
    for images,labels in tqdm(train_data):
        images=images.to(device)
        labels=labels.to(device)
        preds=model(images)
        loss=criterion(preds,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
torch.save(model.state_dict(),os.path.join(save_dir,'info.pth'))
print('model saved')

In [ ]:
accuracy(train_data,model)
accuracy(test_data,model)